In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 75.1 MB/s eta 0:00:00


In [2]:
# Ensure faiss-cpu is installed
!pip install faiss-cpu

# Now import the necessary libraries
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np

docs = ["OpenAI's CEO is Sam Altman", "India's AI mission is AIRAWAT", "Python is a programming language"]
model = SentenceTransformer('all-MiniLM-L6-v2')

doc_embeddings = model.encode(docs, convert_to_tensor=False)
query = "Who is the CEO of OpenAI?"
query_embedding = model.encode(query)

index = faiss.IndexFlatL2(len(query_embedding))
index.add(np.array(doc_embeddings))
D, I = index.search(np.array([query_embedding]), k=1)

print("Retrieved:", docs[I[0][0]])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.9 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Retrieved: OpenAI's CEO is Sam Altman


In [3]:
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np

docs = ["OpenAI's CEO is Sam Altman", "India's AI mission is AIRAWAT", "Python is a programming language"]
model = SentenceTransformer('all-MiniLM-L6-v2')

doc_embeddings = model.encode(docs, convert_to_tensor=False)
query = "What is python?"
query_embedding = model.encode(query)

index = faiss.IndexFlatL2(len(query_embedding))
index.add(np.array(doc_embeddings))
D, I = index.search(np.array([query_embedding]), k=1)

print("Retrieved:", docs[I[0][0]])

Retrieved: Python is a programming language


In [4]:
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np

docs = ["OpenAI's CEO is Sam Altman", "India's AI mission is AIRAWAT", "Python is a programming language"]
model = SentenceTransformer('all-MiniLM-L6-v2')

doc_embeddings = model.encode(docs, convert_to_tensor=False)
query = "What is indias mission?"
query_embedding = model.encode(query)

index = faiss.IndexFlatL2(len(query_embedding))
index.add(np.array(doc_embeddings))
D, I = index.search(np.array([query_embedding]), k=1)

print("Retrieved:", docs[I[0][0]])

Retrieved: India's AI mission is AIRAWAT


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

docs = [
    "OpenAI developed GPT models",
    "Jersey cows give good quality milk",
    "India launched AIRAWAT AI mission"
]

query = "Tell me about OpenAI and GPT"

vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(docs)
query_vector = vectorizer.transform([query])

scores = cosine_similarity(query_vector, doc_vectors)

for i, score in enumerate(scores[0]):
    print(f"Doc {i+1} Score: {score:.4f}")


Doc 1 Score: 0.7071
Doc 2 Score: 0.0000
Doc 3 Score: 0.0000


In [2]:
# ✅ STEP 1: Chunking Function
def chunk_text(text, chunk_size=150, overlap=30):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

# ✅ STEP 2: Load File (adjust path if needed)
with open("/content/conversation_chatgpt.txt", "r", encoding="cp1252") as f:
    full_text = f.read()

chunks = chunk_text(full_text)
print(f"✅ Total Chunks: {len(chunks)}")

# ✅ STEP 3: Load Model and Move to GPU
from sentence_transformers import SentenceTransformer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("🚀 Using device:", device)

model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# ✅ STEP 4: Encode chunks on GPU
chunk_embeddings = model.encode(
    chunks,
    convert_to_tensor=True,
    device=device,
    show_progress_bar=True
)

# ✅ STEP 5: Encode query and search
from sentence_transformers import util

query = "How are you?"
query_embedding = model.encode(
    query,
    convert_to_tensor=True,
    device=device
)

# ✅ STEP 6: Cosine similarity
scores = util.cos_sim(query_embedding, chunk_embeddings)[0]

# ✅ STEP 7: Show top-k results
top_k = 3
top_chunks = scores.topk(top_k)

for score, idx in zip(top_chunks.values, top_chunks.indices):
    print(f"\n🔎 Score: {score:.4f}")
    print(f"📄 Chunk: {chunks[idx]}")


✅ Total Chunks: 44635
🚀 Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1395 [00:00<?, ?it/s]


🔎 Score: 0.3553
📄 Chunk: in my room after breakfast tomorrow will you Yes father Your hands are rather cold Louisa Are you not well Quite well father And cheerful She looked at him again and smiled in her peculiar manner I am as cheerful father as I usually am or usually have been Thats well said Mr Gradgrind So he kissed her and went away and Louisa returned to the serene apartment of the haircutting character and leaning her elbow on her hand looked again at the shortlived sparks that so soon subsided into ashes Are you there Loo said her brother looking in at the door He was quite a young gentleman of pleasure now and not quite a prepossessing one Dear Tom she answered rising and embracing him how long it is since you have been to see me Why I have been otherwise engaged Loo in the evenings and in the

🔎 Score: 0.3454
📄 Chunk: and terrible event. By very slow degrees, and with frequent relapses that alarmed and grieved my friend, I recovered. I remember the first time I became capa

In [6]:
from sentence_transformers import SentenceTransformer, util
import torch

# ✅ Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("🚀 Using device:", device)

# ✅ Load model
model = SentenceTransformer("intfloat/e5-large-v2")  # Better contextual awareness


# ✅ Chunking function
def chunk_text(text, chunk_size=150, overlap=30):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        chunks.append(chunk)
    return chunks

# ✅ Load text file once
with open("/content/conversation_chatgpt.txt", "r", encoding="cp1252") as f:
    full_text = f.read()

chunks = chunk_text(full_text)
print(f"✅ Total Chunks: {len(chunks)}")

# ✅ Encode all chunks once
chunk_embeddings = model.encode(
    chunks, convert_to_tensor=True, device=device, show_progress_bar=True
)

# ✅ Create a retrieval function
def ask_question(query_text):
    query_embedding = model.encode(query_text, convert_to_tensor=True, device=device)
    scores = util.cos_sim(query_embedding, chunk_embeddings)[0]
    top_score, top_index = torch.max(scores, dim=0)

    if top_score < 0.5:
        print(f"\n⚠️ Low confidence (score: {top_score:.4f}). Showing top 3 matches:")
        top_k = 3
        top_chunks = scores.topk(top_k)
        for score, idx in zip(top_chunks.values, top_chunks.indices):
            print(f"\n🔎 Score: {score:.4f}")
            print(f"📄 Chunk:\n{chunks[idx]}")
    else:
        print(f"\n🥇 Top Match Score: {top_score:.4f}")
        print(f"📄 Retrieved Chunk:\n{chunks[top_index]}")




🚀 Using device: cuda


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

✅ Total Chunks: 44635


Batches:   0%|          | 0/1395 [00:00<?, ?it/s]

# USE YOUR OWN **DATASET** ABOVE

In [7]:
# ✅ Example — Just Change This Line!
your_question = "can you tell me about AI and its future?"
ask_question(your_question)


🥇 Top Match Score: 0.8446
📄 Retrieved Chunk:
AI has become in our daily lives? Absolutely! It’s incredible how AI has evolved, especially in areas like voice assistants, recommendation systems, and even healthcare. It feels like every day, there’s a new AI-powered tool that makes life easier. Right, and what amazes me is how seamlessly AI fits into our routines now. I mean, I remember when voice assistants like Siri and Alexa were just novelties, and now they’re pretty much indispensable. I know, right? And it’s not just the voice assistants. AI is helping doctors detect diseases faster, improving car navigation systems, and even predicting weather patterns more accurately. The potential is limitless. For sure. But it does make me wonder about the future. How far will AI go? Will it start making decisions that impact our lives in ways we can’t even predict yet? That’s a great question. I think AI is becoming more autonomous, especially with


In [8]:
your_question = "Do you know time travel?"
ask_question(your_question)


🥇 Top Match Score: 0.8231
📄 Retrieved Chunk:
alternate versions of yourself, or changing history is both thrilling and confusing. Do you think time travel is possible? According to some theories in physics, it might be possible under certain conditions, like with wormholes or extreme speeds close to the speed of light. But we have no practical way of achieving it yet. If you could travel to any point in time, where would you go? I think I’d visit the distant future, maybe 500 years from now, to see how humanity has evolved and what kinds of advancements we’ve made. What’s your take on the idea of parallel universes? It’s a fascinating concept. The idea that there could be infinite versions of our universe, with slight or major differences, opens up endless possibilities. Each decision we make could create a new timeline somewhere. Do you think there could be a universe where things are radically different from our
